#### General
Memory architecture in PostgreSQL can be classified into two broad categories:
- Local memory area – allocated by each backend process for its own use.
- Shared memory area – used by all processes of a PostgreSQL server.

<img src="./helpers/memory.png" alt="drawing" width="700"/>

#### Local Memory Area
Each backend process allocates a local memory area for query processing. The area is divided into several sub-areas, whose sizes are either fixed or variable.

sub-area                |   description
---------               |   -----------
work_mem                |   The executor uses this area for sorting tuples by ORDER BY and DISTINCT operations, and for joining tables by merge-join and hash-join operations.
maintenance_work_mem    |   Some kinds of maintenance operations (e.g., VACUUM, REINDEX) use this area.
temp_buffers            |   The executor uses this area for storing temporary tables.	

#### Shared Memory
A shared memory area is allocated by a PostgreSQL server when it starts up. This area is also divided into several fixed-sized sub-areas.

sub-area                |   description
---------               |   -----------
shared buffer pool      |   PostgreSQL loads pages within tables and indexes from a persistent storage to this area, and operates them directly.
WAL buffer              |   To ensure that no data has been lost by server failures, PostgreSQL supports the WAL mechanism. WAL data (also referred to as XLOG records) are the transaction log in PostgreSQL. The WAL buffer is a buffering area of the WAL data before writing to a persistent storage.
commit log              |   The commit log (CLOG) keeps the states of all transactions (e.g., in_progress, committed, aborted) for the concurrency control (CC) mechanism.


In addition to the shared buffer pool, WAL buffer, and commit log, PostgreSQL allocates several other areas, as shown below:

- Sub-areas for the various access control mechanisms. (e.g., semaphores, lightweight locks, shared and exclusive locks, etc)
- Sub-areas for the various background processes, such as the checkpointer and autovacuum.
- Sub-areas for transaction processing, such as savepoints and two-phase commit.
and others.

#### Using pg_buffercache for memory analysis

In [ ]:
export PGHOST=db
export PGUSER=postgres
export PGDATABASE=postgres

In [ ]:
psql -c "create extension pg_buffercache;"

##### View pg_buffercache
- One row per buffer
- Unused are shown with NULL on all fields except bufferid
- Shared system catalog shown as belonging to DB zero
- Using 128MB (default) of shared_buffers with 8kB of block size, there are 16,384 buffers, so pg_buffercache has the same number of 16,384 rows.


In [ ]:
# Inspect it
psql -c "\d pg_buffercache;"

##### Summary with pg_buffercache_summary

In [ ]:
psql -c "\df pg_buffercache_summary;"
psql -c "SELECT pg_buffercache_summary();"

##### Usage count with pg_buffercache_usage_counts

In [ ]:
psql -c "\df pg_buffercache_usage_counts;"
psql -c "SELECT pg_buffercache_usage_counts();"

##### Some complex queries for cache analysis

In [ ]:
# Get buffer count per relation
psql << EOM
    SELECT n.nspname, c.relname, count(*) AS buffers
    FROM 
        pg_buffercache b JOIN pg_class c 
            ON b.relfilenode = pg_relation_filenode(c.oid) AND b.reldatabase IN (0, (SELECT oid FROM pg_database WHERE datname = current_database()))
        JOIN pg_namespace n ON n.oid = c.relnamespace
    GROUP BY n.nspname, c.relname
    ORDER BY 3 DESC
    LIMIT 10;
EOM

In [ ]:
# Get shared memory cache usage by database
psql << EOM
WITH empty_handled AS (
    SELECT 
        CASE 
            WHEN c.reldatabase IS NULL THEN 'Unused'
            WHEN c.reldatabase = 0 THEN 'System Catalog'
            ELSE d.datname
        END AS database
    FROM 
        pg_buffercache AS c LEFT JOIN pg_database AS d ON c.reldatabase = d.oid
)
    SELECT
        database,
        count(*) AS cached_blocks
    FROM 
        empty_handled
    GROUP BY database
    ORDER BY database;

    SELECT COUNT(*) AS total_chached_blocks FROM pg_buffercache;
EOM

#### Shared memory techniques
- MMAP: Anonymous Memory Mapped Files
- Posix Shared Memory: Shared memory in file style done with temp file system on /dev/shm
- System V shared memory: Shared memory in actual file mapping